## FAKE NEWS DETECTOR - ENSEMBLE LEARNING

### PLAN OF ACTION

- Load the dataset, i.e. combine it into one, and create true / false labels
- Then, utilizing HuggingFace's Transformers library, retrieve the pre-trained BERT model to use as the foundation of a Fake News Detection Model. BERT has incredible language understanding abilities. As a result, it will help the model better grasp news context and hence generate educated predictions about whether news is false or not - The Base Model and general architecture are then defined. For example, we may use PyTorch to define, train, and evaluate neural network models.
- After that, we freeze the weights on the BERT beginning layers. If we do not accomplish this, we will lose all of our past knowledge.
- Next, we add new Trainable Layers. In general, feature extraction layers are the only information from the basic model that we utilize. We must add additional layers on top of the model to forecast its particular jobs. Furthermore, we establish a new output layer because the pre-trained model's ultimate output will almost definitely differ from the result we desire for our model, which is binary 0 and 1.
- The final stage is to fine-tune our model. Making minor tweaks to the model to improve it.
- After that, we use our Fake News Detection Model on unseen data to create predictions.

### Imports

In [ ]:
#### Scikit-learn
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB,GaussianNB
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.base import TransformerMixin
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn import model_selection

from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import GridSearchCV

#### Others
import pandas as pd
import matplotlib.pyplot as plt


### Constants

In [ ]:
DATA_BASE_1_PATH = "./data/ensemble_learning/dataset1/"
DATA_BASE_2_PATH = "./data/ensemble_learning/dataset2/"
NUM_RECORDS_TO_SAMPLE = 3000
TRAIN_RATIO = 0.7
TEST_RATIO = 0.3
PYTORCH_BATCHS_SIZE = 32
TRAINING_EPOCHS = 2 # Number of training epochs

### Load data

The dataset chosen comes from Kaggle: https://www.kaggle.com/datasets/clmentbisaillon/fake-and-real-news-dataset. 

It has two separate .csv files, one having the real news, called "True.csv" and another one, having the fake news, called "Fake.csv". Both files have the exact same data form.

They both include the title of the news, the entire news article text, the subject, which is basically the category of news and the date on which it was published. 

We must now put them together if we are to conduct training and testing.

Let us begin by importing pandas.

The two csv files are converted into pandas dataframes and named.

A categorization column is required to determine if a record is false news or not. We create a new column named classification and set all of its entries to 1 with the command fake['classification'] = 1. Our classifier will detect bogus news and assign it a value of one.

The index counts of the two dataframes are then reset by setting the option ignore_index to True.

In [ ]:
# Load Dataset
true_data = pd.read_csv(DATA_BASE_1_PATH + 'True.csv')
fake_data = pd.read_csv(DATA_BASE_1_PATH + 'Fake.csv')

# Adding classification column
fake_data['classification'] = 1
true_data['classification'] = 0

# Concatenate two dataframes together
true_fake = pd.concat([true_data,fake_data], ignore_index=True)

### Data exploration

The goal of data exploration is to visualize data in order to conduct a more in-depth research, find early insights, and detect trends. We might begin by simply charting the amount of news items in each category:

When we import matplotlib, a new variable called category_dist is created that counts the number of times the numbers 1 and 0 appear within the classification column. Using plt and a series of commands, we can select the size, type of chart, color, labels, title, and whether or not we want the gridlines.

In [ ]:
# Counting values within each classification bucket
category_dist = true_fake['classification'].value_counts()

# Defining chart
plt.figure(figsize=(8,6))
category_dist.plot(kind='bar', color = '#89b4a1')
plt.xlabel("Classification", fontsize = 12)
plt.ylabel("Number of News Headlines by Classification", fontsize = 12)
plt.title("Distribution of News Headlines by Classification", fontsize = 15)
plt.grid(False)

# Generating chart
plt.show()

The dataframe contains around 40.000 records, with somewhat more bogus news. We don't need to use any unbalanced data treatment strategies like upsampling because the difference is so small.

Feeding the model selection pipeline with more than 40.000 records, each of which has 750 words or more, may result in a RAM memory overflow. A easy method is to choose a smaller percentage of the records.

Pandas provides an easy-to-use command that allows the developer to sample a defined number of records while creating a random_state for repeatability of findings. I chose 3000 records since it is an arbitrary amount that represents slightly less than 10% of the dataframe and should not create RAM overflow.

In [ ]:
sample_df = true_fake.sample(NUM_RECORDS_TO_SAMPLE, replace=True, random_state = 44)

In [ ]:
# Counting values within each classification bucket
category_dist_sample = sample_df['classification'].value_counts()

# Defining chart
plt.figure(figsize=(8,6))
category_dist_sample.plot(kind='bar', color = '#89b4a1')
plt.xlabel("Classification", fontsize = 12)
plt.ylabel("Number of News Headlines by Classification", fontsize = 12)
plt.title("Distribution of News Headlines by Classification", fontsize = 15)
plt.grid(False)

# Generating chart
plt.show()

Even with 3000 records, the distribution between fake and truthful news stays the same. This means we can move on to the next phase.

Another dataset only for testing purposes will be used to deploy the model on unseen data. Records were scraped from 244 websites based on the categorization of a Chrome Extension called BS Detector. You can find the dataset at the following link (https://www.kaggle.com/datasets/mrisdal/fake-news). The major difference is that this one contains only fake reviews. It has 21 columns in total, among which the most relevant are:

First, we import genuine and fake csv files into a pandas dataframe. Then, in the 'Target' column, we insert the labels True / Fake. Finally, we use random mixing to combine the two dataframes into a single dataframe.

For the classification problem, for the three datasets, we only need text and information on whether the news is truthful or not correct.

### Preprocessing

Text cleansing is essential in NLP. Stopwords, special characters, and HTML elements all add unnecessary complexity to the processing of our models.

Eliminating all of the above may result in a reduction in training and testing time.

### Train and evaluate (Model Selection)

For the project, I considered 7 common classification models, including parametric and non-parametric ones. All of them are supervised, learning models:

- Linear Discriminant Analysis (LDA)
- k-Nearest-Neighbours (kNN)
- Gaussian Naive Bayes (GaussianNB)
- Logistic regression (LR)
- Decision tree classifier (CART)
- Support Vector Machines (SVM)
- Linear SVM
- Random Forest
- GBDT

It is now time to put the model selection process into action. The code runs data through a count vectorizer and all of the previously mentioned models before doing k-fold cross-validation. After that, the average accuracy is determined and displayed in a boxplot chart. Let's take it one step at a time:


- scikit-learn and matplotlib are these the only two libraries required? Sklearn contains all of the functions required for doing machine learning on data. Every model is imported using a "sub-library": LogisticRegression, KNeighborsClassifier, DecisionTreeClassifier, SVC, GaussianNB, and LinearDiscriminantAnalysis are all examples of classification algorithms.


- At this point, we may specify the input and target variables based on the data. The input variable xis "text," which contains the review corpus; the output variable yis "classification," which displays the labels 0 and 1.

In [ ]:
# Defining input and target variable
x_input = sample_df['text']
y_target = sample_df['classification']

- We know that some of the models under consideration, such as decision tree classifiers, require a dense matrix. Dense matrices have a high proportion of non-zero values. To avoid errors, the ToDenseTransformerclass ensures that all matrices are dense.

In [ ]:
# Adding To Dense Transfomer to satisfy dense data requirement
class ToDenseTransformer(TransformerMixin):
    def transform(self, X, **transform_params):
        return X.todense()
    def fit(self, X, y=None, **fit_params):
        return self

- The list models is then constructed, and each model's object is appended to the list. The list results, on the other hand, will include all of the distinct model scores linked with their names.

In [ ]:
# prepare models
models = []
models.append(('LR', LogisticRegression(max_iter=10000)))
models.append(('LDA', LinearDiscriminantAnalysis()))
models.append(('KNN', KNeighborsClassifier()))
models.append(('CART', DecisionTreeClassifier()))
models.append(('NB', GaussianNB()))
models.append(('SVM', SVC()))

- The k-fold option specifies the number of k-folds desired. The notion of cross-validation is introduced here. Cross-validation tries to improve model accuracy estimation. It is known as k-fold cross validation, where k is the number of sub-groups into which the data is separated. The model is trained on a subset of the data and then tested on the remainder k-1. The accuracy is calculated on the average score.

- The pipeline runs cross-validation on the k-fold of choice and applies the count vectorizer, dense transformer, and model of choice. The findings are then printed to the console.

In [ ]:
# evaluate each model in turn
results = []
names = []
scoring = 'accuracy'
kfold = model_selection.KFold(n_splits=10)

#Building pipeline and gathering results
for name, model in models:
    classifier = Pipeline([('vect', CountVectorizer(strip_accents=None, lowercase = False, preprocessor = None)), ('todense', ToDenseTransformer()), (name, model)])
    cv_results = model_selection.cross_val_score(classifier, x_input, y_target, cv=kfold, scoring=scoring)
    results.append(cv_results)
    names.append(name)
    msg = "%s: Average accuarcy = %f (Variance = %f)" % (name, cv_results.mean(), cv_results.std())
    print(msg)

- Finally, matplotlib creates a boxplot chart to help us analyze the results.

In [ ]:
# boxplot algorithm comparison
fig = plt.figure()
fig.suptitle('Algorithm Comparison')
ax = fig.add_subplot(111)
plt.boxplot(results)
ax.set_xticklabels(names)
plt.show()

A boxplot provides information on five crucial accuracy statistics for our models:

- the bare minimum
- first quartile (25th percentile)
- the average (second quartile)
- third quartile (75th percentile)
- the very best.

All of the models used for cross-validation are plotted on the x-axis. Instead, we have the accuracy score on the y-axis. From the chart alone, it’s hard to tell exactly how well each model performed; for example, the logistic regression and the decision tree classifier are very close to each other

## >>>>>>>>>>>>>>>> NOTE

CART is the best performer in terms of accuracy but the logistic regression is slightly better (by a very thin margin) at delivering consistent results.

### Improving models (Hyperparamenter tuning)

### - Logistic Regression

Logistic regression is one of the two models with the best accuracy and lowest variance, making it an excellent option for hyperparameter optimization.

The logistic regression model's two parameters are as follows:

- A solver is an algorithm that aids the model's adaptation to data. There are five forms of logistic regression: liblinear, lbfgs, newton-cg, sag, and saga.

- The regularization strength is expressed by the C parameter. A larger number for C indicates more regularization. When the model is supplied with previously unknown data, regularization is synonymous with generalization.

The following code optimizes the above hyperparameters as well as the count vectorizer parameters IN ORDER TO FIND THE BEST PARAMETERS FOR COUNT VECTORIZER AND LOGISTIC REGRESSION MODEL:

- Libraries had previously been imported in the previous code cell, but I opted to re-import them to make this code sample self-contained. In this situation, we simply require sklearn with cross-validation, model deployment, pipelining, and count vectorizer sub-packages.

- This time, the pipeline just contains the count vectorizer and the logistic regression model. There is no need for a thick transformer.

- Moving on, the parameters list contains all of the potential hyperparameter combinations that the grid_search tries for each component of the pipeline. The parameter max_df of the Count, Vectorizer, for example, is in charge of the model's generalization.

- A max_df eliminates words that appear too often, and a max_df of 0.7 removes words that appear too frequently, concretely in more than 70% of the documents. In one scenario vect__max_df will combine a max_df of 0.7 with a ngram_range of (1,1), using a kernel poly and a C parameter of 10. The total “fits” (combinations) are 405 because each cross-validation is performed 5 times.

- The final section of code begins reporting the results on the console when calculations are conducted after starting grid_search.fit.

In [ ]:
# Creating a Pipeline with CountVectorizer and LogisticRegression
pipeline = Pipeline([
    ('vect', CountVectorizer()),
    ('LR', LogisticRegression(max_iter = 10000))
])

# Defining hyperparameters
parameters = {
    'vect__max_df':[0.7,0.8,0.9],
    'vect__ngram_range':  [(1,1), (1,2), (1,3)],
    'LR__solver': ['newton-cg', 'lbfgs', 'liblinear'],
    'LR__C': [100, 10, 1.0, 0.1]
}

# Define grid search
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
grid_search = GridSearchCV(pipeline, param_grid=parameters, refit = True, verbose = 3, cv=5)
grid_result = grid_search.fit(x_input.values.astype('U'), y_target.values.astype('U'))

# Summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

- C = 100
- type of kernel = liblinear
- max_df = 0.7
- ngram_range = (1, 3)

With the settings, the model achieves a 98% accuracy when deploying the cross-validation method on only 3000 news. The result is already extremely promising. Given the optimal hyperparameters such as C and max_df, we can already tell the generalization of the algorithm should be enough to correctly classify unseen data.

### - Decision Tree Classifier (CART)

The Decision Tree Classifier (CART) has the highest accuracy of the six methods studied. It should perform even better after hyperparameter adjustment.

The CART model has three parameters:

- max_features is "the number of features to consider each time to make the split decision". If the data collection has 50 columns, you may set this option to 10 to include just 10 of them during the training phase.

- max_depth as the greatest number of "ramifications" a tree can have is specified by max_depth. A greater number may result in overfitting.

- min_samples_leaf is defined as "the minimum number of samples required to be present at a leaf node". The value determines how many split points the tree must have.
There are no additional parameters than hyperparameters.

In [ ]:
# Creating a Pipeline with CountVectorizer and LogisticRegression
pipeline = Pipeline([
    ('vect', CountVectorizer()),
    ('CART', DecisionTreeClassifier())
])

# Defining hyperparameters
parameters = {
    'vect__max_df':[0.7,0.8,0.9],
    'vect__ngram_range':  [(1,1), (1,2), (1,3)],
    'CART__max_features': [0.2, 0.4, 0.6, 0.8],
    'CART__max_depth': [3,4,5,6],
    'CART__min_samples_leaf': [0.04, 0.06, 0.08]
}

# Define grid search
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
grid_search = GridSearchCV(pipeline, param_grid=parameters, refit = True, verbose = 3, cv=5)
grid_result = grid_search.fit(x_input.values.astype('U'), y_target.values.astype('U'))

# Summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

- max_depth = 4
- max_features = 0.8
- min_sample_leaf = 0.04
- max_df = 0.7
- ngram_range = (1, 1)

With the above hyperparameters for the count vectorizer and the algorithm, the average accuracy reaches a 99% score.

### >>>>>>>>>>>>>>>>> NOTE

Once we deploy the model on the 40.000 records and test it on 9000, the accuracy stays the same. A 99% precision remains a great achievement, on the other hand, though, 1% of approximately 4000 records has been misclassified, and the logistic regression seems overall a better solution for deployment on new records.

### Predictions

We've finally arrived at the end. As a reminder of what has been accomplished thus far:

- We determined that logistic regression is the best model.

- A logistic regression model was tweaked, trained on 31.000 data, and tested on the remaining 9000.

- This part seeks to determine whether the preceding hypothesis of a common characteristic within the dataset is correct. How do we go about it? We do this by testing the model on a second fake news dataset to see if it produces the same results. The collection has only one limitation: it only contains bogus news.

In [ ]:
# Creating dataframe
fake_test = pd.read_csv(DATA_BASE_2_PATH + "fake.csv")

# Adding classification column to identify fake-news
fake_test['classification'] = 1

# Selecting relevant columns with only english records
fake_test = fake_test[['text', 'language', 'classification']]
fake_test = fake_test.query("language == 'english'")

# Espliciting input and output variables
x_test = fake_test['text']
y_test = fake_test['classification']

# Transforming 'text' column into bag of words
bow_test = count_vect.transform(x_test.values.astype('U'))

# Classify new records
predicted_LR_test = LR.predict(bow_test)

# Print accuracy report
print(classification_report(y_test, predicted_LR_test))

There might be a number of reasons why a trained model is so accurate, especially in this scenario.

- "Overfitting" is among them. Overfitting is likely to occur if the model classifies records extremely well within the same dataset but loses accuracy when deployed on fresh data.

- A "lucky pull" might be another factor. If a model trains on the correct fraction of a dataset and is tested on easy-to-classify records, a fortunate pull may have occurred. Cross-validation, on the other hand, should eradicate the problem at its source.

- A third explanation might be that the dataset shares a feature that external data does not. It is probable that all of the phony news in this dataset contains the term or the hashtag “fake”. It would be much easier for a model to understand truthful and fake news within the same dataset, but once presented with new data, accuracy might decrease.

A common solution to all of the above would be to deploy the model on a different dataset and assess whether performance stays the same or decreases.

As anticipated, the dataset only contained fake news, and the classifier correctly predicted each one of them, maintaining what had been promised during the tuning and testing phase. Our model classifies with 100% precision whether the news is fake or not, even with an external dataset with minimum computational cost.

Still, considering the algorithm has been deployed on a different dataset, the performance is more than satisfactory. It allows us to eliminate the overfitting and common feature hypotheses.

### Conclusions